# Lab 4.3 - Chatbot to detect emotions

Copyright, Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

In this notebook, we will create a Telegram chatbot that will answer questions, given a predefined question-answer set. 

**Main goal of this notebook**: The most important goal of this notebook is to have a Telegram chatbot that you can ask factual questions to, and receive predefined answers.

**At the end of this notebook, you will**:
* **Use a predefined question - answering dataset**: 

## Using a question-answer dictionary

In [1]:
import json
import random
import pickle
import datetime
import requests

In [2]:
def read_token():
    tokens_path = './data/tokens.json'
    with open(tokens_path) as f:
        tokens = json.load(f)

    return tokens['CLTL_token']

def read_qa():
    qa_path = './data/emotions.json'
    with open(qa_path) as f:
        qa_data = json.load(f)

    return qa_data

In [3]:
class BotHandler:
    def __init__(self, token):
        self.token = token
        self.api_url = "https://api.telegram.org/bot{}/".format(token)

    def get_all_messages(self, offset=None, timeout=200):
        """ Function to get all messages sent to the bot """
        method = 'getUpdates'
        params = {'timeout': timeout, 'offset': offset}
        resp = requests.get(self.api_url + method, params)
        return resp.json()['result']

    def get_last_message_by(self, chat_id):
        """ Function to get the last message sent to the bot by a specific user"""
        messages = self.get_all_messages()
        messages_by_user = list(filter(lambda d: d['message']['chat']['id'] == chat_id, messages))

        if messages_by_user:
            last_message = messages_by_user[-1]['message']['text']
        else:
            last_message = None

        return last_message
    
    def send_message_to(self, chat_id, text):
        """ Function to send a message from the bot to a specific user"""
        params = {'chat_id': chat_id, 'text': text}
        method = 'sendMessage'
        resp = requests.post(self.api_url + method, params)

In [4]:
def classify_emotion(chat, vectorizer, transformer, classifier, label_encoder):
    counts = vectorizer.transform(chat)
    tfidf = transformer.transform(counts)
    predictions = classifier.predict(tfidf)

    for review, predicted_label in zip(chat, predictions):
        predicted_emotion = label_encoder.classes_[predicted_label]
        
    return predicted_emotion

In [5]:
def create_response(message, qa_data, vectorizer, transformer, classifier, label_encoder):
    response = "I cannot respond to this"
    message = [message]
    emotion = classify_emotion(message, vectorizer, transformer, classifier, label_encoder)
    
    for i in qa_data['intents']:
        if emotion == i['category']:
            print("Emotion detected: {emotion}".format(emotion=emotion))
            response = random.choice(i['responses'])
            break
    
    return response

In [6]:
def load_models():
    filename_vectorizer = '../lab3.machine_learning/models/utterance_vec.sav'
    filename_transformer = '../lab3.machine_learning/models/utterance_transf.sav'
    filename_encoder = '../lab3.machine_learning/models/label_encoder.sav'
    filename_classifier = '../lab3.machine_learning/models/svm_linear_clf_bow.sav'

    # load the classifier and the vectorizer from disk
    loaded_classifier = pickle.load(open(filename_classifier, 'rb'))
    loaded_vectorizer = pickle.load(open(filename_vectorizer, 'rb'))
    loaded_transformer = pickle.load(open(filename_transformer, 'rb'))
    loaded_label_encoder = pickle.load(open(filename_encoder, 'rb'))
    
    return loaded_vectorizer, loaded_transformer, loaded_classifier, loaded_label_encoder

In [7]:
CLTL_TOKEN = read_token()
user_id = 408043639

qa_data = read_qa()
vectorizer, transformer, classifier, label_encoder = load_models()

In [8]:
bot = BotHandler(CLTL_TOKEN)
last_message = bot.get_last_message_by(user_id)
response = create_response(last_message, qa_data, vectorizer, transformer, classifier, label_encoder)
bot.send_message_to(user_id, response)


print("Received: {message}".format(message=last_message))
print("Responded: {response}".format(response=response))

Emotion detected: joy
Received: You are so funny
Responded: That's great!


/usr/local/anaconda3/envs/ma-hlt-labs/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


## End of this notebook